In [26]:
import requests
import json
import os
import sys
from dotenv import load_dotenv
load_dotenv()
import PySimpleGUI as sg


In [27]:
def LLM(prompt, system_message, temperature, max_tokens, model,return_json=False):
    url = 'http://localhost:11435/api/passthrough/chat'
    headers = {'Content-Type': 'application/json'}
    payload = {
        'model': model,
        'cache': '1h',
        'keep_alive': '0m',
        'messages': [
            {'role': 'system', 'content': system_message},
            {'role': 'user', 'content': prompt}
        ],
        'temperature': temperature,
        'max_tokens': max_tokens,
        'return_json': return_json
    }

    try:
        response = requests.post(url, headers=headers, json=payload)
        response.raise_for_status()  # Raise an HTTPError if the HTTP request returned an unsuccessful status code
        data = response.json()
        #print('Success:', data)
        return data
    except requests.exceptions.RequestException as error:
        print('Error:', error)
        raise


In [28]:
def askUserWhatTheyWant():
    layout=[[sg.Text('What do you want to do?')],
            [sg.Multiline('', key='prompt', size=(50, 10))],
            [sg.Button('Generate'), sg.Button('Exit')]]
    window = sg.Window('Choose action', layout)
    while True:
        event, values = window.read()
        if event == sg.WIN_CLOSED or event == 'Exit':
            break
        if event == 'Generate':
            window.close()
            print
            return values['prompt']
    window.close()
    return None

In [29]:
def show_output(output):
    layout = [[sg.Text(output)]]
    window = sg.Window('Output', layout)
    event, values = window.read()
    window.close()

In [ ]:
import re
def extractPython(text):
    # extract from ```python to ``` or ``` or end of text
    # return the extracted text
    match = re.search(r'```python(.*?)```', text, re.DOTALL)
    if match:
        return match.group(1)
    match = re.search(r'```(.*?)```', text, re.DOTALL)
    if match:
        return match.group(1)
    return text

In [30]:
def askUserIfCodeShouldRun(code):
    # show code to user and ask if it should run it, make the window pretty big
    layout = [[sg.Multiline(code, size=(80, 20), key='code')],
            [sg.Button('Run'), sg.Button('Cancel')]]
    window = sg.Window('Code', layout)
    event, values = window.read()
    window.close()
    if event == 'Run':
        old = sys.stdout
        old_e = sys.stderr
        sys.stdout = open('./jail/coder/output.txt', 'w')
        sys.stderr = sys.stdout
        try:
            exec(code)
        except Exception as e:
            show_output(e)        
        sys.stdout.close()
        sys.stdout = old
        sys.stderr = old_e
        with open('./jail/coder/output.txt') as f:
            output = f.read()
            show_output(output)
    else:
        print("Code not executed")
        raise Exception("Code not executed")

In [31]:
def incremental_dump(response):
    path = './jail/coder/'
    if os.path.exists(path) == False:
        os.makedirs(path)
    index = 0
    while os.path.exists(f'{path}{index}.py'):
        index += 1
    with open(f'{path}{index}.py', 'w') as f:
        f.write(response)
    return f'{path}{index}.py'

In [32]:
system_message=[i.strip() for i in """
    You only write python code, please code what the user wishes to do as completely as possible.
    Large Responses are acceptable
""".splitlines()].join('\n')

reply = LLM(askUserWhatTheyWant(), system_message, 0.2, 30000, 'llama3.1',return_json=False)
show_output(reply['message']['content'])

code = extractPython(reply['message']['content'])
incremental_dump(code)
askUserIfCodeShouldRun(code)

NameError: name 'tk' is not defined